<a href="https://colab.research.google.com/github/LucasLessa1/challengeBain/blob/main/Desafio_Bain_%26_Company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1 - Imports**
First, we import the libraries that will be used throughout the code.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
# import gdown

# **2 - Pre-Processing**

In [2]:
# #download data csv
# def download(id):
#   url = 'https://drive.google.com/uc?id=' + str(id)
#   gdown.download(url, output = None, quiet = False)


# download('1ysfB1yjSXUVn-pxo2Y5mlnNJo6d9JdjG')

Below, you can note that we used the "delimiter" parameter to separate the fields. Note also that the we renamed the destinated_area field, removing the extra comma.

In [3]:
df = pd.read_csv('data/historical-database.csv', delimiter=";")
df.rename(columns={'destinated_area':'destinated_area'}, inplace = True)
df

,year,city_code,product_type,product,"destinated_area,"
0,01/01/1974,960b4f2c94a2fb2c,temporary,Others,"0,"
1,01/01/1974,746cc42bfb8f6b62,temporary,Others,"0,"
2,01/01/1974,6cce2bf873870afc,temporary,Others,"0,"
3,01/01/1974,4de42e351006a2ae,temporary,Others,"0,"
4,01/01/1974,5b6072f8f6d37acc,temporary,Others,"0,"
...,...,...,...,...,...
73739,01/01/2013,74d75dde6dc4a5ec,pasture,Livestock,"150866,5644"
73740,01/01/2014,74d75dde6dc4a5ec,pasture,Livestock,"152446,396"
73741,01/01/2015,74d75dde6dc4a5ec,pasture,Livestock,"154681,0205"
73742,01/01/2016,74d75dde6dc4a5ec,pasture,Livestock,"164706,2778"


When we import the data, all data types were designated as objects (mostly mixed numeric and non-numeric data type).

In [4]:
df.dtypes

year                object
city_code           object
product_type        object
product             object
destinated_area,    object
dtype: object

But to do some treatment, we change the data type to string.

In [5]:
df = df.astype(str)

We noticed that some values were just commas, so we just interpreted them as empty, replacing them with NaN (not a number).

In [6]:
df.loc[df['destinated_area'] == ",", 'destinated_area'] = np.nan
df['destinated_area'].isna().sum()

KeyError: 'destinated_area'

For now on, we continued to treat weird values, such as "0," and changing commas to dots.

In [ ]:
df['destinated_area'] = df['destinated_area'].replace(['0,'], '0')
destinated_area = df['destinated_area'].to_list()
destinated_area = [
    item if isinstance(item, float) else item.replace(",", ".")
    for item in destinated_area
]
df['destinated_area'] = destinated_area

Then, we change some data types to float (when that should be the case), or to date values.

In [ ]:
df['destinated_area'] = df['destinated_area'].astype(float)

In [ ]:
column_mean = df["destinated_area"].mean()
df['destinated_area'] = df['destinated_area'].fillna(column_mean)

In [ ]:
(df == 0).sum()

In [ ]:
df['date'] = pd.to_datetime(df['year'], format="%d/%m/%Y")

In [ ]:
df['year'] = pd.DatetimeIndex(df['date']).year

The final result of the treatment is the below dataframe and data types.

In [ ]:
df.dtypes

In [ ]:
df

# **3 - Importing other databases**

The .csv file provided by Bain contains only the date, city_code, product_type, product and destination_area fields, one of which is a hash. The group understands that this file is sufficient to make predictions. However, in search of a more accurate result, it would be interesting to aggregate other data sources in the bain dataframe. So it was done.

In this section, data on **livestock**, **weather**, **humidity** and **products** from Paraná are added to the dataframe.

In [ ]:
download("1uQnV90A0SAFT1KxwjjUOBX4W9zkbZ-Vd") #https://drive.google.com/file/d/1uQnV90A0SAFT1KxwjjUOBX4W9zkbZ-Vd/view?usp=sharing
download('1RII2akbU72zngbRdUpFGIakxcmRoqTPN') #https://drive.google.com/file/d/1RII2akbU72zngbRdUpFGIakxcmRoqTPN/view?usp=sharing
download('1EY28_qJTSq8e0JHEfbeADPSkFCp2kV-T') #https://drive.google.com/file/d/1EY28_qJTSq8e0JHEfbeADPSkFCp2kV-T/view?usp=sharing

In [ ]:
df_products = pd.read_csv("/content/products_final.csv")
df_met = pd.read_csv("/content/Meteorologia.csv")
df_umidade = pd.read_csv("/content/umidade.csv")

We start treating the new product data to our main dataframe.

In [ ]:
# Droping unnecessary fields.
del df_products['Quantidade produzida (mil frutos)']
del df_products['Rendimento médio da produção (mil frutos por Hectare)']
del df_products["index"]

Next, we create a dictionary to unite the main dataframe product types (from Bain) with the new product data.

In [ ]:
di = {
    "Arroz": "Rice", "Cacau": "Cocoa", "Soja": "Soy", "Feijão":"Beans", 
    "Coco":"Others", "Mandioca":"Cassava","Milho":"Corn","Maracujá":"Others", 
    "Café":"Others", "Borracha":"Others" ,"Mamão":"Others", "Limão":"Others", 
    "Algodão arbóreo":"Others", "Algodão herbáceo":"Others" ,"Sorgo":"Sorghum",
    "Cacau":"Cocoa", "Dendê":"Palm oil", "Açaí":"Açaí", "Outros":"Others"
}
df_products['Produto'].replace(di, inplace=True)

In [ ]:
df_products = df_products[df_products['Produto'] != "Others"]
df_products['Produto'].value_counts()

In [ ]:
index = []
x, y = df_met.shape

for i in range(0, x, 1):
    index.append(i)

df_met.insert(0, "Unnamed: 0", index, allow_duplicates=False)

Here, we merge both meteorology and Bain dataframes by the year fileds. For such, we will first create a year column, and then merge.

In [ ]:
df_met['date'] = pd.to_datetime(df_met['DATA..YYYY.MM.DD.'], format="%Y/%m/%d")
df_met['DATA..YYYY.MM.DD.'] = pd.DatetimeIndex(df_met['date']).year
df_met.rename(columns={'DATA..YYYY.MM.DD.':'Ano'}, inplace=True)

In [ ]:
df_met = df_met.replace(-9999.0, np.NaN)
df_met = df_met.groupby('Ano', as_index=False).mean()
df_met.rename(columns={'Ano':'year'}, inplace=True)

In [ ]:
df_merge = df_umidade.merge(df_products, on="Ano", how='left')
df_merge.rename(columns={'Ano':'year'}, inplace=True)

df_merge = df_merge.merge(df_met, on="year", how='left')
df_merge = df_merge.merge(df, on="year", how='left')

pred_2018 = df_merge[df_merge['year'] == 2018]
pred_2019 = df_merge[df_merge['year'] == 2019]

Deleting unnecessary fields, renaming others.

In [ ]:
del pred_2018['Unnamed: 0_x']
del pred_2018['Unnamed: 0_y']
del pred_2018['X']
del pred_2018['date']
del pred_2018['product']

del pred_2019['Unnamed: 0_x']
del pred_2019['Unnamed: 0_y']
del pred_2019['X']
del pred_2019['date']
del pred_2019['product']

pred_2018 = pred_2018.drop(["city_code", "destinated_area"], axis=1)
pred_2019 = pred_2019.drop(["city_code", "destinated_area"], axis=1)
pred_2018.rename(columns={'Produto':'product'}, inplace=True)
pred_2019.rename(columns={'Produto':'product'}, inplace=True)

Adding the products witch is not in our dataset.

In [ ]:
df.groupby("product")["product_type"].unique()

In [ ]:
# 2018
new_others = pd.Series(data={"product":"Others", "product_type":"temporary"}, name='10')
new_livestock = pd.Series(data={"product":"Livestock", "product_type":"pasture"}, name='11')

pred_2018 = pred_2018.append(new_others, ignore_index=True)
pred_2018 = pred_2018.append(new_livestock, ignore_index=True)


# 2019
new_others = pd.Series(data={"product":"Others", "product_type":"temporary"}, name='10')
new_livestock = pd.Series(data={"product":"Livestock", "product_type":"pasture"}, name='11')

pred_2019 = pred_2019.append(new_others, ignore_index=True)
pred_2019 = pred_2019.append(new_livestock, ignore_index=True)

In [ ]:
pred_2018.fillna(pred_2018.mean(), inplace=True)
pred_2019.fillna(pred_2019.mean(), inplace=True)

Merging original dataframe with aditional datasets

In [ ]:
df_products.rename(columns={'Ano':'year', 'Produto':'product'}, inplace=True)

In [ ]:
df_with_products = df.merge(df_products, on=["year", "product"], how="left")

In [ ]:
df_total = df_with_products.merge(df_met, on="year", how="left")

In [ ]:
del df_total['Unnamed: 0_x']
del df_total['Unnamed: 0_y']
del df_total['X']
del df_total['date']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

# Perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df_total[['product', "product_type"]]).toarray())

# Merge one-hot encoded columns back with original DataFrame
final_df = df_total.join(encoder_df)

final_df

In [ ]:
final_df = final_df.rename(columns={
    13: 'product type: temporary', 12: 'product type: permanent',
    11:'product type: pasture', 10:'soy', 9:'sorghum', 8:'rice', 
    7:'palm oil', 6:'others', 5:'livestock', 4:'corn', 3:'cocoa', 
    2:'cassava', 1:'beans', 0:'açaí'
}
)

Here, we will perform One Hot Enconding. One hot encoding can be defined as the essential process of converting the categorical data variables to be provided to machine and deep learning algorithms which in turn improve predictions as well as classification accuracy of a model. Here, we are applying it on "team" column.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

# Perform one-hot encoding on 'team' column
encoder_df = pd.DataFrame(encoder.fit_transform(
                          pred_2018[['product', "product_type"]]).toarray()
                          )

# Merge one-hot encoded columns back with original DataFrame
pred_2018 = pred_2018.join(encoder_df)

encoder_df = pd.DataFrame(encoder.fit_transform(
                          pred_2019[['product', "product_type"]]).toarray()
                          )

pred_2019 = pred_2019.join(encoder_df)

In [ ]:
pred_2019 = pred_2019.rename(columns={
    13: 'product type: temporary', 12: 'product type: permanent',
    11:'product type: pasture',10:'soy', 9:'sorghum', 8:'rice',7:'palm oil', 6:'others',
    5:'livestock',4:'corn',3:'cocoa', 2:'cassava', 1:'beans',0:'açaí'
}
)

In [ ]:
pred_2018 = pred_2018.rename(columns={
    13: 'product type: temporary', 12: 'product type: permanent',
    11:'product type: pasture',10:'soy', 9:'sorghum', 8:'rice',7:'palm oil', 6:'others',
    5:'livestock',4:'corn',3:'cocoa', 2:'cassava', 1:'beans',0:'açaí'
    })

In [ ]:
df_total = final_df

In [ ]:
# columns = df_total.columns.drop(["city_code", "product_type", "product"])

In [ ]:
# Removing Nan values
df_total = df_total.dropna()

In [ ]:
df_total = df_total[df_total['year'] >= 2000]
df_total = df_total[df_total['year'] <= 2017]

# **4 - Implementing XGBoost**

NVIDIA defines XGBoost as such: "XGBoost, which stands for Extreme Gradient Boosting, is a scalable, distributed gradient-boosted decision tree (GBDT) machine learning library. It provides parallel tree boosting and is the leading machine learning library for regression, classification, and ranking problems."


In this section, we will seek to apply XGBoost to our final dataset, which comprises both Bain data and external data that the group sought.

https://debuggercafe.com/a-guide-to-xgboost-in-python/

https://xgboost.readthedocs.io/en/stable/python/python_intro.html

https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/

https://medium.com/towards-data-science/xgboost-predicting-life-expectancy-with-supervised-learning-1f61e0222f5f

We decided to use XGBoost because it gained significant favor in the last few years as a result of winning Kagle competitions on regression and prediction data. 

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Here, we are defining all the features that we could use in our algorithm. Although, not all are gonna be used.

In [ ]:
label = df_total["destinated_area"]
features = df_total.drop([
    "destinated_area", "city_code", "product_type",
    "product", "year"], axis=1)

In [ ]:
features.columns


It is important to sort the features in the same order.

In [ ]:
columns_total = features.columns
pred_2018 = pred_2018.reindex(columns=columns_total)
pred_2019 = pred_2019.reindex(columns=columns_total)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

min_max_scaler = preprocessing.MinMaxScaler()

scaler = StandardScaler()

In [ ]:
df_scaler = scaler.fit_transform(features)

In [ ]:
df_scaler = pd.DataFrame(df_scaler, columns=columns_total)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_scaler, label, random_state=42, test_size=0.2)

Below is the selection of the features that presents better fit to the results that we want. For that, it is good to plot the correlation of the features. Although, we assume that the readers are a little familirized with this type of algorithm. 

In [ ]:
# Correlation
corrmat = df_total.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20, 20))

# Plot heat map
g = sns.heatmap(df_total[top_corr_features].corr(), annot=True, cmap="RdYlGn")

Below, we are applying XGBoost with 10.000 estimators, a learning rate of 10% and 4 jobs.

In [ ]:

xgbr = xgb.XGBRegressor(
    n_estimators=10000, learning_rate=0.1,
    n_jobs=4)

In [ ]:
xgbr.fit(
    x_train, y_train,
    early_stopping_rounds=5, eval_set=[(x_test, y_test)],
    verbose=False)

The training score obtained is presented below.

In [ ]:
score = xgbr.score(x_train, y_train)
print("Training score: ", score)

In [ ]:
from sklearn.metrics import mean_squared_error

It is also important to measure the obtained error. It is presented below.

In [ ]:
y_pred = xgbr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
score = xgbr.score(x_test, y_test)  
score

In [ ]:
df_scaler.tail()

In [ ]:
# pred_2018 = pred_2018.drop(["product", "year"], axis=1)
# pred_2019 = pred_2019.drop(["product", "year"], axis=1)
pred_2018.columns

In [ ]:
pred_2019.columns

In [ ]:
len(x_train), len(x_test)

In [ ]:
len(pred_2019)

In [ ]:
df["city_code"].value_counts()

In [ ]:
y_pred_2018 = xgbr.predict(pred_2018)
y_pred_2019 = xgbr.predict(pred_2019)


In [ ]:
y_pred_2018

In [ ]:
y_pred_2019

Unfortunately, the results obtained by applying the XGBOOST algorithm did not turn out as expected. Thus, we chose to apply a Linear Regression to the Bain database, without considering the extra databases initially collected by the team.

# **5 - Implementing Linear Regression**


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

# perform one-hot encoding on 'team' column
encoder_df = pd.DataFrame(encoder.fit_transform(df[
    ["city_code", 'product', "product_type"]]).toarray())

# merge one-hot encoded columns back with original DataFrame
final_df = df.join(encoder_df)

final_df

In [ ]:
final_df.dtypes

In [ ]:
col_orden = final_df.drop([
    "destinated_area", "product",
    "product_type", "city_code",
    "date"], axis=1).columns

In [ ]:
x = final_df.drop([
    "destinated_area", "product",
    "product_type", "city_code", "date"], axis=1)
y = final_df["destinated_area"]

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x = np.array(x.values.tolist())
y = np.array(y.values.tolist())

model = LinearRegression(normalize=True, n_jobs=-1)

In [ ]:
model = LinearRegression().fit(x,y)

In [ ]:
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope: \n', model.coef_)

In [ ]:
download("1ZQg8VSDpIey3agzQSsuHoj95wctrRWWD") #https://drive.google.com/file/d/1ZQg8VSDpIey3agzQSsuHoj95wctrRWWD/view?usp=sharing

In [ ]:
pred_2018_2019 = pd.read_csv("/content/submission-template.csv")
copy = pred_2018_2019

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore')

# perform one-hot encoding on 'team' column
encoder_df = pd.DataFrame(encoder.fit_transform(pred_2018_2019[
    ["city_code", 'product', "product_type"]]).toarray())

# merge one-hot encoded columns back with original DataFrame
pred_df = pred_2018_2019.join(encoder_df)

pred_df

In [ ]:
pred_df['date'] = pd.to_datetime(pred_df['year'], format="%Y-%d-%m")
pred_df['year'] = pd.DatetimeIndex(pred_df['date']).year


In [ ]:
pred_df = pred_df.reindex(columns=col_orden)

In [ ]:
pred_df

In [ ]:
# pred_2018_2019 = pred_df.drop(["destinated_area", "product", "product_type", "city_code", "date", "Unnamed: 0"], axis=1)
pred_2018_2019 = pred_df

In [ ]:
pred_2018_2019.shape

In [ ]:
x.shape

In [ ]:
linear_pred_2018_2018 = model.predict(pred_2018_2019)

In [ ]:
linear_pred_2018_2018

In [ ]:
pred_2018_2019

In [ ]:
copy

In [ ]:
copy["destinated_area"] = linear_pred_2018_2018
del copy["Unnamed: 0"]

In [ ]:
copy

The cell below is in comment format, since its command is to download the final submission file.

In [ ]:
# from google.colab import files
# copy.to_csv('DG2Z8G.csv') 
# files.download('DG2Z8G.csv')